## 언어 판정
- 어떤 언어로 작성됐는지 판정하는 것

## 머신러닝으로 언어 판정 해보기
- 문자를 구성하는 글자로 판정

### 언어 판정에 Unicode 코드 포인트 사용
- Unicode 코드 포인트를 인덱스로 하는 배열을 만든다. (0~65535(FFFF) 등)
- 배열의 각 요소는 대응하는 Unicode 코드 포인트의 출현 빈도로 초기값은 0
- 문자 내부의 각 문자를 Unicode 코드 포인트로 변환하고 출현 빈도값을 더함

### 알고리즘 선택하기
- 텍스트 데이터의 경우 나이브 베이즈(Naive Bayes) 추천

### 사용하는 문자가 다른 언어 판정하기

In [3]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Unicode 코드 포인트 출현 빈도 판정
def count_codePoint(str):
    # Unicode 코드 포인트를 저장할 배열
    counter = np.zeros(65535)
    
    for i in range(len(str)):
        # 각 문자를 Unicode 코드 포인트로 변환
        code_point = ord(str[i])
        if code_point > 65535:
            continue
        # 출현 횟수 세기
        counter[code_point] += 1
    
    # 각 요소를 문자 수로 나눠 정규화
    counter = counter / len(str)
    return counter

# 학습 전용 데이터 준비하기
ko_str = '이것은 한국어 문장입니다.'
ja_str = 'これは日本語の文章です。'
en_str = 'This is English Sentences.'

x_train = [count_codePoint(ko_str), count_codePoint(ja_str), count_codePoint(en_str)]
y_train = ['ko', 'ja', 'en']

# 학습
clf = GaussianNB()
clf.fit(x_train, y_train)

# 평가 전용 데이터 준비하기
ko_test_str = '안녕하세요'
ja_test_str = 'こんにちは'
en_test_str = 'Hello'

x_test = [count_codePoint(ko_test_str), count_codePoint(ja_test_str), count_codePoint(en_test_str)]
y_test = ['ko', 'ja', 'en']

# 평가
y_pred = clf.predict(x_test)
print(y_pred)
print('정답률 :', accuracy_score(y_test, y_pred))

['ko' 'ja' 'en']
정답률 : 1.0


### 사용하는 문자가 같은 언어 판정하기

In [4]:
%%time

import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import glob

# Unicode 코드 포인트 출현 빈도 판정
def count_codePoint(str):
    # Unicode 코드 포인트를 저장할 배열
    counter = np.zeros(65535)
    
    for i in range(len(str)):
        # 각 문자를 Unicode 코드 포인트로 변환
        code_point = ord(str[i])
        if code_point > 65535:
            continue
        # 출현 횟수 세기
        counter[code_point] += 1
    
    # 각 요소를 문자 수로 나눠 정규화
    counter = counter / len(str)
    return counter

# 학습 데이터
x_train = []
y_train = []

for file in glob.glob('../datasets/lang/train/*.txt'):
    # 언어 정보를 추출하고 레이블로 저장
    y_train.append(file[23:25])
    
    # 파일 내부의 문자열을 모두 추출한 뒤 빈도 배열로 변환, 입력데이터로 사용
    file_str = ''
    for line in open(file, 'r', encoding='utf8'):
        file_str = file_str + line
    x_train.append(count_codePoint(file_str))

# 학습
clf = GaussianNB()
clf.fit(x_train, y_train)

# 평가 데이터
x_test = []
y_test = []

for file in glob.glob('../datasets/lang/test/*.txt'):
    # 언어 정보를 추출하고 레이블로 저장
    y_test.append(file[22:24])
    
    # 파일 내부의 문자열을 모두 추출한 뒤 빈도 배열로 변환, 입력데이터로 사용
    file_str = ''
    for line in open(file, 'r', encoding='utf8'):
        file_str = file_str + line
    x_test.append(count_codePoint(file_str))

# 평가
y_pred = clf.predict(x_test)
print(y_pred)
print('정답률 :', accuracy_score(y_test, y_pred))

['de' 'en' 'es']
정답률 : 1.0
Wall time: 344 ms
